Documents - Text objects, within a corpus of documents
Document -> Node structure -> Structure indices
https://www.youtube.com/watch?v=wYZJq8CNmTw
https://colab.research.google.com/drive/1JOzbVzrm8_GJAmuh2Qcjsxf5Rg0yK3AG?usp=sharing#scrollTo=4OHSwkDySV_-

In [1]:
import nest_asyncio
nest_asyncio.apply()

import logging
import sys

In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
# !pip install -qU llama-index

In [4]:
# !pip install wikipedia llama-index-readers-wikipedia # Collect semantic data from Wikipedia

In [5]:
# !pip install -qU llama-index-vector-stores-qdrant qdrant-client #  vector database today will be powered by QDrant 

In [6]:
# !pip install -q -U sqlalchemy # Quantitative data

In [7]:
# !pip install -qU llama-index-callbacks-wandb

In [8]:
# !pip install wandb==0.16.6

In [9]:
# !pip install protobuf==3.20

In [10]:
import os
import getpass
import wandb
PROJECT = "mi-eval-llm"

In [11]:
from dotenv import find_dotenv, load_dotenv

# _ = load_dotenv(find_dotenv())
load_dotenv()

True

In [12]:
wandb.login(anonymous="allow")
run = wandb.init(project=PROJECT, job_type="agent")

# protobuf==3.20
# wandb==0.16.6

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: gowri-apollo (apollo-messaging-intelligence). Use `wandb login --relogin` to force relogin


In [13]:
WANDB_KEY = ""

In [14]:
os.environ["WANDB_API_KEY"] = getpass.getpass("WandB API Key: ")

WandB API Key:  ········


In [15]:
import llama_index
from llama_index.core import set_global_handler

set_global_handler("wandb", run_args={"project": PROJECT})
wandb_callback = llama_index.core.global_handler

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


### Task 3: Settings

In [16]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

In [17]:
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

### Index Creation using VectorStore
Vector store index takes documents and splits them into nodes. Then it creates vector embeddings of the text of every node - now the nodes are ready to be queried.

- Use wikipedia reader from llama_index and load the pages
- Use locally hostable QDrant vector store

In [18]:
from llama_index.readers.wikipedia import WikipediaReader 

movie_list = [
    "Dune (2021 film)",
    "Dune: Part Two",
    "The Lord of the Rings: The Fellowship of the Ring",
    "The Lord of the Rings: The Two Towers",
]

wiki_docs = WikipediaReader().load_data(pages=movie_list, auto_suggest=False)

In [19]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, models


client = QdrantClient(location=":memory:")
client.create_collection(
    collection_name="movie_wikis",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

True

Create vector store and storage context -> allow us to create an empty vector store index that will be used to add nodes

In [20]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

vector_store = QdrantVectorStore(client=client, collection_name="movie_wikis")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    [], storage_context=storage_context,
)

wandb: Logged trace tree to W&B.


### Node Construction
Loop through the documents and metadata and construct nodes


In [21]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor

pipeline = IngestionPipeline(transformations=[TokenTextSplitter()])

for movie, wiki_doc in zip(movie_list, wiki_docs):
    nodes = pipeline.run(documents=[wiki_doc])
    for node in nodes:
        node.metadata = {"title": movie}
    index.insert_nodes(nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


### Simple RAG - QueryEngine


In [22]:
simple_rag = index.as_query_engine()

In [23]:
for k, v in simple_rag.get_prompts().items():
    print(v.get_template())
    print("\n~~~~~~~~~~~~~~~~~~\n")

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 

~~~~~~~~~~~~~~~~~~

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 

~~~~~~~~~~~~~~~~~~



In [24]:
response = simple_rag.query("Who is the evil Wizard in the story?")
response.response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


'Saruman the White is the evil wizard in the story.'

In [25]:
response = simple_rag.query("Who are the giant beings that roam across the world?")
response.response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


'The giant beings that roam across the world are the sandworms.'

In [26]:
print([x.metadata["title"] for x in response.source_nodes])

['Dune (2021 film)', 'The Lord of the Rings: The Fellowship of the Ring']


### Auto Retriever Functional Tool
To select correct metadata filter and query the filtered index. Nodes with desired metadata.

- Create a vector store info object to hold the relevant metadata for each title

In [27]:
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3

vector_store_info = VectorStoreInfo(
    content_info="semantic information about movies",
    metadata_info=[
        MetadataInfo(
            name="title",
            type="str",
            description='title of the movie, one of ["Dune (2021 film)", "Dune: Part Two", "The Lord of the Rings: The Fellowship of the Ring", "The Lord of the Rings: The Two Towers"]'
        )
    ]
)

In [28]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

In [29]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
): 
    query = query or "Query"
    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)
    responses = query_engine.query(query)

    return str(response)

In [30]:
description = f"""\
Use this tool to look up non-review based information about films.
The vector database schema is given below:
{vector_store_info.json()}
"""

In [31]:
description

'Use this tool to look up non-review based information about films.\nThe vector database schema is given below:\n{"metadata_info": [{"name": "title", "type": "str", "description": "title of the movie, one of [\\"Dune (2021 film)\\", \\"Dune: Part Two\\", \\"The Lord of the Rings: The Fellowship of the Ring\\", \\"The Lord of the Rings: The Two Towers\\"]"}], "content_info": "semantic information about movies"}\n'

In [32]:
auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn,
    name="semantic-film-info",
    description=description,
    fn_schema=AutoRetrieveModel
)

In [33]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(
    tools=[auto_retrieve_tool],
    verbose=True,
)

In [34]:
response = agent.chat("Who starred in the 2021 film?")
print(str(response))

Added user message to memory: Who starred in the 2021 film?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: semantic-film-info with args: {"query":"cast of Dune (2021 film)","filter_key_list":["title"],"filter_value_list":["Dune (2021 film)"]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Got output: The giant beings that roam across the world are the sandworms.

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling F

wandb: Logged trace tree to W&B.


I apologize for the inconvenience. It seems there was an issue retrieving the cast information for the 2021 film "Dune." Let me try to gather the cast details again.


In [35]:
response = agent.chat("Who are those giant guys from Lord of the Rings that roam around the forest?")
print(str(response))

Added user message to memory: Who are those giant guys from Lord of the Rings that roam around the forest?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
Retrying llama_index.llms.openai.base.OpenAI._chat in 0.7121588826004072 seconds as it raised BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_eUe08KJB3t0nqCF4EzSUCJk9", 'type': 'invalid_request_error', 'param': 'messages.[14].role', 'code': None}}.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.2124521560453674 seconds as

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_eUe08KJB3t0nqCF4EzSUCJk9", 'type': 'invalid_request_error', 'param': 'messages.[14].role', 'code': None}}

### Quantitative RAG Pipeline with NL2SQL Tooling

In [ ]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune1.csv

In [ ]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune2.csv

In [ ]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/lotr_fotr.csv

In [ ]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/lotr_tt.csv

In [36]:
import pandas as pd

dune1 = pd.read_csv("./dune1.csv")
dune2 = pd.read_csv("./dune2.csv")
lotr_fotr = pd.read_csv("./lotr_fotr.csv")
lotr_tt = pd.read_csv("./lotr_tt.csv")

In [37]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:")

In [38]:
dune1.to_sql("Dune (2021 film)", engine)
dune2.to_sql(
  "Dune: Part Two",
  engine
)
lotr_fotr.to_sql(
  "The Lord of the Rings: The Fellowship of the Ring",
  engine
)
lotr_tt.to_sql(
  "The Lord of the Rings: The Two Towers",
  engine
)

149

In [39]:
from llama_index.core import SQLDatabase
sql_database = SQLDatabase(
    engine=engine,
    include_tables=movie_list
)

In [40]:
from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=movie_list,
)

In [41]:
DESCRIPTION = """This tool should be used to answer any and all review related inquiries by translating a natural language query into a SQL query with access to tables:
'Dune (2021 film)' - containing info. about the first movie in the Dune series,
'Dune: Part Two'- containing info. about about the second movie in the Dune series,
'The Lord of the Rings: The Fellowship of the Ring' - containing info. about the first movie in the Lord of the Ring series,
'The Lord of the Rings: The Two Towers' - containing info. the second movie in the Lord of the Ring series,
"""

In [42]:
from llama_index.core.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql-query",
    description=DESCRIPTION
)

In [43]:
agent = OpenAIAgent.from_tools(
    tools=[sql_tool],
    verbose=True
)

In [44]:
response = agent.chat("What is the average rating of the 2nd Lord of the Rings movie?")
response.response

Added user message to memory: What is the average rating of the 2nd Lord of the Rings movie?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: sql-query with args: {"input":"average rating of 'The Lord of the Rings: The Two Towers'"}
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'Dune (2021 film)' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'Dune: Part Two' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'The Lord of the Rings: The Fellowship of the Ring' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT

wandb: Logged trace tree to W&B.


'The average rating of the 2nd Lord of the Rings movie, "The Lord of the Rings: The Two Towers," is 9.18 out of 10.'

In [45]:
response = agent.chat("What movie series has better reviews, Lord of the Rings or Dune?")
response.response

Added user message to memory: What movie series has better reviews, Lord of the Rings or Dune?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: sql-query with args: {"input": "average rating of 'The Lord of the Rings: The Fellowship of the Ring'"}
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'Dune (2021 film)' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'Dune: Part Two' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'The Lord of the Rings: The Fellowship of the Ring' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Re

wandb: Logged trace tree to W&B.


'The average rating of "The Lord of the Rings" series is higher than that of the "Dune" series. \n- "The Lord of the Rings: The Fellowship of the Ring" has an average rating of approximately 9.87 out of 10.\n- "Dune (2021 film)" has an average rating of 8.34 out of 10.'

### Combined RAG Pipeline

In [46]:
combined_tool_agent = OpenAIAgent.from_tools(
    tools=[auto_retrieve_tool, sql_tool],
    verbose=True
)

In [47]:
response = combined_tool_agent.chat("Which movie is about a ring, and what is the average rating of the movie?")
response.response

Added user message to memory: Which movie is about a ring, and what is the average rating of the movie?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: semantic-film-info with args: {"query":"movie about a ring","filter_key_list":["title"],"filter_value_list":["The Lord of the Rings: The Fellowship of the Ring","The Lord of the Rings: The Two Towers"]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Got output: The average rating of "The Lord of the Rings" series is higher than that of the "Dune" series. 
- "The Lord of the Rings: The Fellowship 

wandb: Logged trace tree to W&B.


'The movie about a ring is "The Lord of the Rings: The Fellowship of the Ring." It has an average rating of approximately 9.87 out of 10.'

In [48]:
response = combined_tool_agent.chat("What worlds do the LoTR, and Dune movies take place in?")
response.response

Added user message to memory: What worlds do the LoTR, and Dune movies take place in?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: semantic-film-info with args: {"query": "worlds in LoTR movies", "filter_key_list": ["title"], "filter_value_list": ["The Lord of the Rings: The Fellowship of the Ring", "The Lord of the Rings: The Two Towers"]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Got output: The movie about a ring is "The Lord of the Rings: The Fellowship of the Ring." It has an average rating of approximately 9.87 out of 10.

=== Call

wandb: Logged trace tree to W&B.


'I apologize for the inconvenience. It seems there was an issue retrieving the information about the worlds in which the "The Lord of the Rings" and "Dune" movies take place. Let me try once more to provide you with the correct information.'

In [49]:
wandb_callback.finish()